In [1]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import sys
import os

PROJECT_DIRECTORY = os.path.join(os.getcwd(), '..')
sys.path.append(PROJECT_DIRECTORY)

from src.utils import *

In [2]:
abt_treino = pd.read_parquet(PROJECT_DIRECTORY + '/data/processed/abt_treino.parquet')

# Metadados da Base

In [3]:
metadados = gerar_metadados(abt_treino,['id','safra'],'y')
metadados

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
0,VAR_62,Explicativa,6465,78.74,18,float64
1,VAR_70,Explicativa,6369,77.57,649,float64
2,VAR_61,Explicativa,5897,71.82,554,float64
3,VAR_49,Explicativa,5662,68.96,669,float64
4,VAR_36,Explicativa,5622,68.47,627,float64
5,VAR_63,Explicativa,5587,68.04,1427,float64
6,VAR_43,Explicativa,5544,67.52,103,float64
7,VAR_12,Explicativa,5544,67.52,996,float64
8,VAR_18,Explicativa,5480,66.74,267,float64
9,VAR_21,Explicativa,5480,66.74,1696,float64


# Remoção de variáveis com altas taxas de nulos

In [4]:
# Limiar para corte de percentual de nulos
THRESHOLD_PC_NULOS = 60

metadados_nulos_filtrados = metadados[metadados['PC_NULOS'] < THRESHOLD_PC_NULOS]

print(f'Quantidade de variáveis iniciais: {metadados.shape[0]}')
print(f'Quantidade de variáveis após filtragem dos nulos: {metadados_nulos_filtrados.shape[0]}')

Quantidade de variáveis iniciais: 81
Quantidade de variáveis após filtragem dos nulos: 62


# Identificando variáveis categóricas

In [5]:
TARGET = ['y']
AUXILIARES = ['id','safra']

# Limiar para classificação como categórica
THRESHOLD_CARDINALIDADE = 25

NUM_VARS = metadados_nulos_filtrados[(metadados_nulos_filtrados['CARDINALIDADE'] > THRESHOLD_CARDINALIDADE) & (metadados_nulos_filtrados['USO_FEATURE'] == 'Explicativa')]['FEATURE'].to_list()
CAT_VARS = metadados_nulos_filtrados[(metadados_nulos_filtrados['CARDINALIDADE'] <= THRESHOLD_CARDINALIDADE) & (metadados_nulos_filtrados['USO_FEATURE'] == 'Explicativa')]['FEATURE'].to_list()

print(f'Quantidade de Variáveis Númericas: {len(NUM_VARS)}')
print(f'Quantidade de variáveis categóricas: {len(CAT_VARS)}')

Quantidade de Variáveis Númericas: 44
Quantidade de variáveis categóricas: 15


# Remoção de variáveis altamente correlacionadas - Variáveis Númericas 

In [6]:
THRESHOLD_CORRELACAO = 0.9

pares_corr, colunas_remover = tratar_multicolinearidade(abt_treino[NUM_VARS], limiar = THRESHOLD_CORRELACAO)

print("Pares com alta correlação:")
print(pares_corr)

print("\nColunas sugeridas para remoção (baseado em nulos e cardinalidade):")
print(colunas_remover)

Pares com alta correlação:
    Variável 1 Variável 2  Correlação
932     VAR_22     VAR_19    0.975977
354     VAR_10     VAR_69    0.975103
855     VAR_24     VAR_58    0.974130
568     VAR_39     VAR_45    0.972208
280     VAR_26     VAR_14    0.954847

Colunas sugeridas para remoção (baseado em nulos e cardinalidade):
['VAR_10', 'VAR_19', 'VAR_26', 'VAR_58', 'VAR_39']


In [7]:
NUM_VARS_V2 = [x for x in NUM_VARS if x not in colunas_remover]
print(f'Quantidade de Variáveis Númericas: {len(NUM_VARS_V2)}')

Quantidade de Variáveis Númericas: 39


# Remoção de variáveis com baixa variância - Variáveis Númericas 

In [8]:
# Remoção de variaveis com um limiar de variância abaixo de 0,05
THRESHOLD_VARIANCIA = 0.05

selector = VarianceThreshold(threshold=THRESHOLD_VARIANCIA)
df_filtrado = abt_treino[NUM_VARS_V2]
df_filtrado = df_filtrado.loc[:, selector.fit(df_filtrado).get_support()]

print(f'Quantidade de Variáveis Númericas: {df_filtrado.shape[1]}')

Quantidade de Variáveis Númericas: 39


# Imputação dos nulos - Variáveis Númericas 

In [9]:
imputer = SimpleImputer(strategy='mean')
abt_treino[NUM_VARS_V2] = imputer.fit_transform(abt_treino[NUM_VARS_V2])

IMPUTER_PATH = PROJECT_DIRECTORY + "/models/artefacts/num_imputer.pickle"

# Salvar o imputer treinado com pickle
with open(IMPUTER_PATH, 'wb') as f:
    pickle.dump(imputer, f)

# Normalização - Variáveis Númericas

In [10]:
scaler = StandardScaler()
abt_treino[NUM_VARS_V2] = scaler.fit_transform(abt_treino[NUM_VARS_V2])

SCALER_PATH = PROJECT_DIRECTORY + "/models/artefacts/num_scaler.pickle"

# Salvar o imputer treinado com pickle
with open(SCALER_PATH, 'wb') as f:
    pickle.dump(scaler, f)

# Colunas duplicadas - Variáveis Categóricas

In [11]:
colunas_duplicadas = encontrar_colunas_duplicadas(abt_treino)
print(colunas_duplicadas)

set()


# Remoção de variáveis altamente correlacionadas - Variáveis Categóricas 

In [12]:
THRESHOLD_CORRELACAO_CAT = 0.9

pares_corr, colunas_remover = tratar_multicolinearidade_cat(abt_treino,CAT_VARS, limiar=THRESHOLD_CORRELACAO_CAT)

print("Pares com alta correlação:")
print(pares_corr)

print("\nColunas sugeridas para remoção (baseado em nulos e cardinalidade):")
print(colunas_remover)

Pares com alta correlação:
   Variável 1 Variável 2  Correlação
27     VAR_74     VAR_78    0.985973
0      VAR_71     VAR_73    0.979719
89     VAR_44     VAR_64    0.925134
94     VAR_40     VAR_64    0.911909

Colunas sugeridas para remoção (baseado em nulos e cardinalidade):
['VAR_40', 'VAR_74', 'VAR_71', 'VAR_44']


In [13]:
CAT_VARS_V2 = [x for x in CAT_VARS if x not in colunas_remover]
print(f'Quantidade de Variáveis Categóricas: {len(CAT_VARS_V2)}')

Quantidade de Variáveis Categóricas: 11


# Imputação dos nulos - Variáveis Categóricas 

In [14]:
cat = abt_treino[[*CAT_VARS_V2,*TARGET]]
metadados_nulos_filtrados[metadados_nulos_filtrados['FEATURE'].isin(CAT_VARS_V2)]

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
26,VAR_73,Explicativa,4354,53.03,11,float64
30,VAR_78,Explicativa,4354,53.03,15,float64
59,VAR_3,Explicativa,309,3.76,12,float64
60,VAR_2,Explicativa,309,3.76,16,float64
61,VAR_4,Explicativa,309,3.76,13,float64
65,VAR_28,Explicativa,40,0.49,13,float64
70,VAR_25,Explicativa,40,0.49,13,float64
76,VAR_20,Explicativa,0,0.00,10,int64
77,VAR_32,Explicativa,0,0.00,9,float64
79,VAR_60,Explicativa,0,0.00,20,float64


In [15]:
THRESHOLD_HN = 30

CAT_VARS_V2_HN = [col for col in 
              metadados_nulos_filtrados[(metadados_nulos_filtrados['FEATURE'].isin(CAT_VARS_V2)) &
                                        (metadados_nulos_filtrados['PC_NULOS'] >= THRESHOLD_HN)
                                        ]['FEATURE']]

CAT_VARS_V2_LN = [coluna for coluna in CAT_VARS_V2 if coluna not in CAT_VARS_V2_HN]

INPUTS_LN = {col: abt_treino[col].mode(dropna=True)[0] for col in CAT_VARS_V2_LN}
INPUTS_HN = {col: 9999 for col in abt_treino[CAT_VARS_V2_HN].columns}

INPUTS_CAT = {**INPUTS_LN,**INPUTS_HN}

In [16]:
# Aplicar a imputação nas variáveis
for col, valor in INPUTS_CAT.items():
    abt_treino[col] = abt_treino[col].fillna(valor)

In [17]:
# Salvando os valores para tratamento de nulos das variáveis categóricas
IMPUTER_CAT_PATH = PROJECT_DIRECTORY + "/models/artefacts/cat_imputer.pickle"

with open(IMPUTER_CAT_PATH, 'wb') as f:
    pickle.dump(INPUTS_CAT, f)

# Dumming - Variáveis categóricas

In [18]:
one_hot_enc = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore',sparse_output=False),
    CAT_VARS_V2),
    remainder='drop')

dados_one_hot = one_hot_enc.fit_transform(abt_treino)
dados_one_hot = pd.DataFrame(dados_one_hot,
                     columns=one_hot_enc.get_feature_names_out(),
                     index=abt_treino.index)
dados_one_hot

# Salvando os valores para tratamento de nulos das variáveis categóricas
ONE_HOT_PATH = PROJECT_DIRECTORY + "/models/artefacts/cat_encoder.pickle"

with open(ONE_HOT_PATH, 'wb') as f:
    pickle.dump(one_hot_enc, f)

# Salvando Colunas Numéricas e Categóricas

In [19]:
NUM_VARS_PATH = PROJECT_DIRECTORY + '/models/artefacts/num_vars.pickle'
CAT_VARS_PATH = PROJECT_DIRECTORY + '/models/artefacts/cat_vars.pickle'
TARGET_VARS_PATH = PROJECT_DIRECTORY + '/models/artefacts/target_vars.pickle'
AUX_VARS_PATH = PROJECT_DIRECTORY + '/models/artefacts/aux_vars.pickle'

with open(NUM_VARS_PATH,'wb') as f:
    pickle.dump(NUM_VARS_V2,f)

with open(CAT_VARS_PATH,'wb') as f:
    pickle.dump(CAT_VARS_V2,f)  

with open(TARGET_VARS_PATH,'wb') as f:
    pickle.dump(TARGET,f)  

with open(AUX_VARS_PATH,'wb') as f:
    pickle.dump(AUXILIARES,f)  

# Função de Tratamento Final

In [20]:
def tratamento_completo(df, aux_vars_path,target_var_path,num_vars_path,cat_vars_path,cat_imputer_path,num_imputer_path,cat_encoder_path,num_scaler_path):
    """
    Aplica um pipeline completo de tratamento de dados a um DataFrame, incluindo seleção de variáveis,
    imputação de nulos, codificação de variáveis categóricas e normalização de variáveis numéricas.

    Parâmetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada com os dados brutos.

    aux_vars_path : str
        Caminho para o arquivo `.pickle` contendo a lista de variáveis auxiliares.

    target_var_path : str
        Caminho para o arquivo `.pickle` contendo a lista de variáveis alvo.

    num_vars_path : str
        Caminho para o arquivo `.pickle` contendo a lista de variáveis numéricas.

    cat_vars_path : str
        Caminho para o arquivo `.pickle` contendo a lista de variáveis categóricas.

    cat_imputer_path : str
        Caminho para o arquivo `.pickle` contendo o dicionário de imputação de variáveis categóricas.

    num_imputer_path : str
        Caminho para o arquivo `.pickle` contendo o imputador de variáveis numéricas.

    cat_encoder_path : str
        Caminho para o arquivo `.pickle` contendo o codificador para variáveis categóricas.

    num_scaler_path : str
        Caminho para o arquivo `.pickle` contendo o normalizador para variáveis numéricas.

    Retorna
    -------
    pandas.DataFrame
        DataFrame tratado com variáveis imputadas, codificadas e normalizadas, pronto para modelagem.
    """

    with open(aux_vars_path,'rb') as f:
        aux_vars = pickle.load(f)

    with open(target_var_path,'rb') as f:
        target_vars = pickle.load(f)
    
    with open(num_vars_path,'rb') as f:
        num_vars = pickle.load(f)

    with open(cat_vars_path,'rb') as f:
        cat_vars = pickle.load(f)

    with open(cat_imputer_path,'rb') as f:
        cat_imputer = pickle.load(f)

    with open(num_imputer_path,'rb') as f:
        num_imputer = pickle.load(f)

    with open(cat_encoder_path,'rb') as f:
        cat_encoder = pickle.load(f)

    with open(num_scaler_path,'rb') as f:
        num_scaler = pickle.load(f)  

    # Selecionando Colunas 
    df = df.reindex(columns=[*aux_vars,
                             *target_vars,
                             *num_vars,
                             *cat_vars])
    
    
    # Imputando Nulos
    df[num_vars] = num_imputer.transform(df[num_vars])
    
    for col, valor in cat_imputer.items():
        df[col] = df[col].fillna(valor)
   
    # Dummy encoding
    dados_one_hot = cat_encoder.transform(df)
    dados_one_hot = pd.DataFrame(dados_one_hot,
                     columns=cat_encoder.get_feature_names_out(),
                     index=df.index)
    
    df = pd.concat([df,dados_one_hot],axis=1)
    df = df.drop(columns=cat_vars)

    # Normalização
    df[num_vars] = num_scaler.transform(df[num_vars])

    
    return df

In [21]:
df_teste = pd.read_parquet(PROJECT_DIRECTORY + '/data/processed/abt_treino.parquet')

In [22]:
aux_vars_path    = PROJECT_DIRECTORY + '/models/artefacts/aux_vars.pickle'
target_var_path  = PROJECT_DIRECTORY + '/models/artefacts/target_vars.pickle'
num_vars_path    = PROJECT_DIRECTORY + '/models/artefacts/num_vars.pickle'
cat_vars_path    = PROJECT_DIRECTORY + '/models/artefacts/cat_vars.pickle'

cat_imputer_path = PROJECT_DIRECTORY + '/models/artefacts/cat_imputer.pickle'
num_imputer_path = PROJECT_DIRECTORY + '/models/artefacts/num_imputer.pickle'

cat_encoder_path = PROJECT_DIRECTORY + '/models/artefacts/cat_encoder.pickle'
num_scaler_path  = PROJECT_DIRECTORY + '/models/artefacts/num_scaler.pickle'


In [23]:
df_teste = tratamento_completo(df = df_teste,
                               aux_vars_path    = aux_vars_path,
                               target_var_path  = target_var_path,
                               num_vars_path    = num_vars_path,
                               cat_vars_path    = cat_vars_path,
                               cat_imputer_path = cat_imputer_path,
                               num_imputer_path = num_imputer_path,
                               cat_encoder_path = cat_encoder_path,
                               num_scaler_path  = num_scaler_path)

df_teste.head()

,id,safra,y,VAR_27,VAR_67,VAR_37,VAR_29,VAR_42,VAR_46,VAR_51,VAR_14,VAR_66,VAR_77,VAR_23,VAR_69,VAR_31,VAR_13,VAR_45,VAR_15,VAR_52,VAR_11,VAR_35,VAR_34,VAR_8,VAR_59,VAR_76,VAR_7,VAR_38,VAR_17,VAR_5,VAR_24,VAR_65,VAR_54,VAR_53,VAR_1,VAR_6,VAR_72,VAR_22,VAR_33,VAR_30,VAR_9,VAR_57,onehotencoder__VAR_73_0.0,onehotencoder__VAR_73_1.0,onehotencoder__VAR_73_2.0,onehotencoder__VAR_73_3.0,onehotencoder__VAR_73_4.0,onehotencoder__VAR_73_5.0,onehotencoder__VAR_73_6.0,onehotencoder__VAR_73_7.0,onehotencoder__VAR_73_8.0,onehotencoder__VAR_73_9.0,onehotencoder__VAR_73_10.0,onehotencoder__VAR_73_9999.0,onehotencoder__VAR_78_0.0,onehotencoder__VAR_78_1.0,onehotencoder__VAR_78_2.0,onehotencoder__VAR_78_3.0,onehotencoder__VAR_78_4.0,onehotencoder__VAR_78_5.0,onehotencoder__VAR_78_6.0,onehotencoder__VAR_78_7.0,onehotencoder__VAR_78_8.0,onehotencoder__VAR_78_9.0,onehotencoder__VAR_78_10.0,onehotencoder__VAR_78_11.0,onehotencoder__VAR_78_12.0,onehotencoder__VAR_78_14.0,onehotencoder__VAR_78_16.0,onehotencoder__VAR_78_9999.0,onehotencoder__VAR_3_0.0,onehotencoder__VAR_3_1.0,onehotencoder__VAR_3_2.0,onehotencoder__VAR_3_3.0,onehotencoder__VAR_3_4.0,onehotencoder__VAR_3_5.0,onehotencoder__VAR_3_6.0,onehotencoder__VAR_3_7.0,onehotencoder__VAR_3_8.0,onehotencoder__VAR_3_9.0,onehotencoder__VAR_3_10.0,onehotencoder__VAR_3_11.0,onehotencoder__VAR_2_0.0,onehotencoder__VAR_2_1.0,onehotencoder__VAR_2_2.0,onehotencoder__VAR_2_3.0,onehotencoder__VAR_2_4.0,onehotencoder__VAR_2_5.0,onehotencoder__VAR_2_6.0,onehotencoder__VAR_2_7.0,onehotencoder__VAR_2_8.0,onehotencoder__VAR_2_9.0,onehotencoder__VAR_2_10.0,onehotencoder__VAR_2_11.0,onehotencoder__VAR_2_12.0,onehotencoder__VAR_2_13.0,onehotencoder__VAR_2_14.0,onehotencoder__VAR_2_17.0,onehotencoder__VAR_4_0.0,onehotencoder__VAR_4_1.0,onehotencoder__VAR_4_2.0,onehotencoder__VAR_4_3.0,onehotencoder__VAR_4_4.0,onehotencoder__VAR_4_5.0,onehotencoder__VAR_4_6.0,onehotencoder__VAR_4_7.0,onehotencoder__VAR_4_8.0,onehotencoder__VAR_4_9.0,onehotencoder__VAR_4_10.0,onehotencoder__VAR_4_11.0,onehotencoder__VAR_4_12.0,onehotencoder__VAR_28_0.0,onehotencoder__VAR_28_1.0,onehotencoder__VAR_28_2.0,onehotencoder__VAR_28_3.0,onehotencoder__VAR_28_4.0,onehotencoder__VAR_28_5.0,onehotencoder__VAR_28_6.0,onehotencoder__VAR_28_7.0,onehotencoder__VAR_28_8.0,onehotencoder__VAR_28_9.0,onehotencoder__VAR_28_10.0,onehotencoder__VAR_28_11.0,onehotencoder__VAR_28_12.0,onehotencoder__VAR_25_0.0,onehotencoder__VAR_25_1.0,onehotencoder__VAR_25_2.0,onehotencoder__VAR_25_3.0,onehotencoder__VAR_25_4.0,onehotencoder__VAR_25_5.0,onehotencoder__VAR_25_6.0,onehotencoder__VAR_25_7.0,onehotencoder__VAR_25_8.0,onehotencoder__VAR_25_9.0,onehotencoder__VAR_25_10.0,onehotencoder__VAR_25_11.0,onehotencoder__VAR_25_12.0,onehotencoder__VAR_20_3,onehotencoder__VAR_20_4,onehotencoder__VAR_20_5,onehotencoder__VAR_20_6,onehotencoder__VAR_20_7,onehotencoder__VAR_20_8,onehotencoder__VAR_20_9,onehotencoder__VAR_20_10,onehotencoder__VAR_20_11,onehotencoder__VAR_20_12,onehotencoder__VAR_32_-0.151023621869443,onehotencoder__VAR_32_-0.10340492822954,onehotencoder__VAR_32_-0.0386138327488513,onehotencoder__VAR_32_-0.0115735952255897,onehotencoder__VAR_32_0.0200194164511392,onehotencoder__VAR_32_0.0467093416827228,onehotencoder__VAR_32_0.0571981886428283,onehotencoder__VAR_32_0.090634138151029,onehotencoder__VAR_32_0.133832912727673,onehotencoder__VAR_60_-0.690749137734589,onehotencoder__VAR_60_-0.411786729965823,onehotencoder__VAR_60_-0.354107585360257,onehotencoder__VAR_60_-0.30845333657335,onehotencoder__VAR_60_-0.231735369185606,onehotencoder__VAR_60_-0.133343746407103,onehotencoder__VAR_60_-0.0737564789495467,onehotencoder__VAR_60_-0.059298901120992,onehotencoder__VAR_60_-0.0304781166421273,onehotencoder__VAR_60_-0.016208722718011,onehotencoder__VAR_60_0.079926420595367,onehotencoder__VAR_60_0.156301087833759,onehotencoder__VAR_60_0.219380082347677,onehotencoder__VAR_60_0.242879678877368,onehotencoder__VAR_60_0.319424920164188,onehotencoder__VAR_60_0.357323965312407,o